# 새로운 노래
* 전 주에 없던 새로운 노래 확인

## 라이브러리

In [212]:
import pandas as pd
import os

## 1주일 동안 새로 진입한 곡 평균

In [214]:
# f_path = 'c:/data/Wantreez/Crawling/music'
# try:
#     os.chdir(f_path)
# except:
#     pass

In [215]:
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/data/Wantreez/Crawling/music'

In [216]:
# 전 주 파일 날짜(일주일)
start_date = '2022-04-25'
end_date = '2022-05-01'

In [217]:
# 비교할 파일 날짜(일주일)
new_start_date = '2022-05-02'
new_end_date = '2022-05-08'
new_dates_df = pd.date_range(new_start_date, new_end_date).astype(str)

In [224]:
# 음원 사이트
site_list = ['flo', 'bugs', 'genie', 'melon', 'vibe']
for site_name in site_list:
    try:
        folder_path = f'/crawled_data/mod_{site_name}/'
        path = code_path + folder_path
        try:
            os.chdir(path)
        except:
            pass

        # 파일 이름들
        file_list = os.listdir()

        # 전체 df 생성
        total_df = pd.DataFrame()
        for one in file_list:
            temp = pd.read_excel(one)
            total_df = pd.concat([total_df, temp], ignore_index=True)
        total_df['날짜'] = total_df['날짜'].astype(str)

        # 전주, 이번주 df
        before_df = total_df[ (total_df['날짜'] >= start_date) & (total_df['날짜'] <= end_date)].reset_index(drop=True)
        new_df = total_df[ (total_df['날짜'] >= new_start_date) & (total_df['날짜'] <= new_end_date)].reset_index(drop=True)

        # 전주 노래 목록
        title_list = before_df['곡'].unique()
        # 이번주 새로 진입한 곡 df
        pop_title_list = []
        pop_artist_list = []
        pop_date_df = pd.DataFrame()
        for new_one_date in new_dates_df:
            new_date_df = total_df[total_df['날짜'] == new_one_date].reset_index(drop=True)

            for i in range(new_date_df.shape[0]):
                one_title = new_date_df['곡'][i]
                one_artist = new_date_df['가수'][i]
                if one_title not in title_list:
                    pop_title_list.append(one_title)
                    pop_artist_list.append(one_artist)

            for two_title, two_artist in zip(pop_title_list, pop_artist_list):
                temp = new_df[ (new_df['곡'] == two_title) & (new_df['가수'] == two_artist)]
                pop_date_df = pd.concat([pop_date_df, temp], ignore_index=True)
                pop_date_df = pop_date_df.drop_duplicates(['날짜', '곡', '가수'], ignore_index=True)

        # 새로 진입한 곡의 평균
        mean_pop_df = pop_date_df.groupby(['소속사', '곡', '가수']).agg('mean').reset_index()
        mean_pop_df.columns = ['소속사', '곡', '가수', '순위평균']
        mean_pop_df['순위평균'] = round(mean_pop_df['순위평균'], 1)

        # 파일 저장
        save_folder = f'/crawled_data/outlier_data/{new_start_date}_{new_end_date}/{site_name}/'
        path = code_path + save_folder
        file_name = f'[{site_name}]{new_start_date}_{new_end_date}_pop_df.xlsx'
        mean_pop_df.to_excel(path + file_name, index=False, encoding='utf-8')
        print(f'{file_name} 출력완료.')
    except:
        pass

[bugs]2022-05-02_2022-05-08_pop_df.xlsx 출력완료.
[genie]2022-05-02_2022-05-08_pop_df.xlsx 출력완료.
[melon]2022-05-02_2022-05-08_pop_df.xlsx 출력완료.
[vibe]2022-05-02_2022-05-08_pop_df.xlsx 출력완료.


# 코드 마지막